In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.errors import DuplicateColumn
conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

def get_data(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
    return data

def get_data_dict(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
        col_name = [col.name for col in cur.description]
    return [ dict(zip(col_name,d)) for d in data]

def get_colname(table):
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM {} LIMIT 1;'.format(table))
        data = [col.name for col in cur.description]
    return data

def exec_transaction(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
    conn.commit()


In [ ]:
import mysql.connector
conn2 = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'honomara',
    password =  'honomara',
    database = 'honomara',
)
cur2 = conn2.cursor(dictionary=True)

In [ ]:
exec_transaction('''CREATE SEQUENCE IF NOT EXISTS after_id_seq ;''')
try:
    exec_transaction('''ALTER TABLE after ADD COLUMN after_id integer NOT NULL DEFAULT nextval('after_id_seq');''')
except DuplicateColumn as err:
    print(err)
    conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

In [ ]:
create_restaurants_table = """
CREATE TABLE IF NOT EXISTS restaurants (
    restaurant_id INT PRIMARY KEY AUTO_INCREMENT,
    restaurant_name VARCHAR(64) NOT NULL,
    place VARCHAR(20),
    comment text
);
"""

cur2.execute(create_restaurants_table)
conn2.commit()

In [ ]:
restaurants = get_data_dict('''
SELECT restaurant,site,avg(cost_max) AS cost_max,
avg(cost_min) AS cost_min , avg(number_max) AS n_max,
avg(number_min) AS n_min 
FROM after 
WHERE restaurant != '?' AND restaurant != '？' AND restaurant != ''
GROUP BY (restaurant,site) 
ORDER BY restaurant;
''')

cur2.execute("INSERT INTO restaurants (restaurant_name,place,comment) VALUES ('Undefined','Undefined','Undefined');")
cur2.execute("UPDATE restaurants SET restaurant_id = 0 WHERE  restaurant_name = 'Undefined' and place = 'Undefined' and comment = 'Undefined';")
for restaurant in restaurants:
    restaurant_name = restaurant['restaurant']
    if restaurant_name == '' or restaurant_name == '?' or restaurant_name == '？':
        continue
    else:
        print(restaurant_name)
    place = restaurant['site']
    comment  = "費用目安 {}\n".format(int((restaurant['cost_max'] +  restaurant['cost_min']) /2))
    comment += "適正人数 {}人\n".format(int( (restaurant['n_max'] +  restaurant['n_min']) /2))
    cur2.execute('INSERT INTO restaurants (restaurant_name,place,comment) VALUES (%s,%s,%s)',(restaurant_name,place,comment))
conn2.commit()

In [ ]:
create_afters_table = """
CREATE TABLE IF NOT EXISTS afters (
    `after_id` INT PRIMARY KEY AUTO_INCREMENT,
    `date` date NOT NULL,
    `after_stage` INT NOT NULL DEFAULT 1,  
    `restaurant_id` INT NOT NULL, 
    `total` INT NOT NULL,
    `title` VARCHAR(128) NOT NULL, 
    `comment` TEXT,
    FOREIGN KEY (`restaurant_id`) REFERENCES restaurants(`restaurant_id`),
    INDEX USING BTREE(`date`));
"""

cur2.execute(create_afters_table)
conn2.commit()

In [ ]:
for data in get_data_dict('SELECT * FROM after ORDER BY date;'):
    name = data['restaurant']
    if name == '' or name == '?' or name == '？':
        restaurant_id = 0
    else:
        cur2.execute('SELECT * FROM restaurants WHERE restaurant_name = %s AND place = %s',(data['restaurant'],data['site']))
        restaurant_id = cur2.fetchall()[0]['restaurant_id']
    after_id = data['after_id']
    date = data['date']
    after_stage=data['after']
    title = data['topic']
    total = data['total']
    comment = data['comment']
    cur2.execute('''
    INSERT INTO afters (after_id,date,after_stage,restaurant_id,total,title,comment) 
    VALUES (%s,%s,%s,%s,%s,%s,%s);''',(after_id,date,after_stage,restaurant_id,total,title,comment))

conn2.commit()

## miscellaneous info

In [ ]:
get_colname('after')